# Organizzazione primo anno AFAM
Corsi di studio: chitarra, basso, batteria, pianoforte, canto.

In [1]:
import pandas as pd
import random 
from datetime import datetime, timedelta

from bs4 import BeautifulSoup
import requests

In [2]:
random.seed(42)

In [3]:
names_path = r'C:\Users\Lucrezia\Documents\GitHub\CPM_exam_organization\dataset\names.csv'
names = pd.read_csv(names_path, header=None)

In [4]:
def split_name_surname(names_surnames):
    name_surname_list = []
    for sn in names_surnames:
        name_surname_list.append(sn.split())

    length_list = [len(ns) for ns in name_surname_list]
    count_length_diff_from_2 = count = sum(1 for x in length_list if x != 2)

    while count_length_diff_from_2 != 0:
        for i,l in enumerate(length_list):
            if l != 2:
                name_surname3 = name_surname_list[i]
                name_surname2 = [name_surname3[0], name_surname3[1]+' '+name_surname3[2]]
                name_surname_list[i] = name_surname2

        new_length_list = [len(ns) for ns in name_surname_list]
        count_length_diff_from_2 = sum(1 for x in new_length_list if x != 2)

    return name_surname_list

## Creazione tabella studenti

In [5]:
student_names = split_name_surname(names[0][:50])
students = pd.DataFrame()
students['id'] = ['st_'+str(i) for i in range(len(student_names))]
students['nome'] = [l[0] for l in student_names]
students['cognome'] = [l[1] for l in student_names]
courses = ['canto', 'chitarra', 'basso', 'pianoforte', 'batteria']
courese_percentage = [0.4, 0.25, 0.15, 0.05, 0.15]
students['corso'] = random.choices(courses, weights= courese_percentage, k=len(student_names))
students.head()

,id,nome,cognome,corso
0,st_0,Simone,Longo,chitarra
1,st_1,Andrea,D'Angelo,canto
2,st_2,Giuseppe,Fontana,canto
3,st_3,Tommaso,Leone,canto
4,st_4,Sofia,Esposito,basso


## Creazione tabella professori

In [6]:
url_docenti = 'https://www.cpm.it/corsi-musica-milano-docenti'
html = requests.get(url_docenti)
soup = BeautifulSoup(html.text, 'html5lib')

box_docenti = soup.find('div', class_='row m-y-lg row-docente')
list_url_docenti = ['https://www.cpm.it'+a['href'] for a in box_docenti.find_all('a')]
list_names = [a.find('h3').text.split('\xa0')  for a in box_docenti.find_all('a')]
list_courses = [a.find('h4').text.replace('\t','').replace('\n','')  for a in box_docenti.find_all('a')]

In [7]:
list_url_docenti[0]

'https://www.cpm.it/docente/1097/corsi-musica-milano-docente-andrea-di-cesare'

In [9]:
# list_corsi_di_insegnamento = []

# for url in list_url_docenti:

#     html = requests.get(url)
#     soup = BeautifulSoup(html.text, 'lxml')
#     for i,bool in enumerate(['Didattica' in h2.text for h2 in soup.find_all('h2')]):
#         if bool:
#             didattica = soup.find_all('h2')[i]
#             next = didattica.find_next('b').text

#     # corsi_di_insegnamento = didattica.find_next('b').text
#     list_corsi_di_insegnamento.append(next)
    
# list_corsi_di_insegnamento

In [87]:
insegna = []
for url in list_url_docenti:
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')

    if soup.find('div', class_='col-md-12 text-docenti'):
        text_docenti = soup.find('div', class_='col-md-12 text-docenti')
        if text_docenti.find_all('b'):
            last_b = text_docenti.find_all('b')[-1]

    if text_docenti.find(string=lambda text: 'Didattica' in text):
        didattica = text_docenti.find(string=lambda text: 'Didattica' in text)

        courses = []
        while didattica.find_next('b'):
            b = didattica.find_next('b')
            courses.append(b.text)
            if b == last_b:
                break
            else:
                didattica = b
        insegna.append(courses)

In [102]:
list_insegna = []
for row in insegna:
    for course in row:
        list_insegna.append(course)

In [110]:
omitted_string = ['Lorenzo Cazzaniga', 'un esame di ammissione ', 'www',
                  'giovedì 21 luglio.', 'giovedì 21 luglio','Segreteria Didattica','corsi@cpm.it','02.641146.1',
                  'P','Corso Accademico di Pianoforte e Tastiere Pop Rock','Filippo Rinaldo', 'Giuliano Lecis', 
                  'Giovanni Boscariol', 'Fabio Nuzzolese', 'Massimo Colombo','Vice Direttore Didattico', 'Responsabile ',
                  'Iscrizioni entro giovedì 16 maggio.', 'completo, altamente specializzante e riconosciuto dal MUR',
                  'Iscriviti ora all’esame d’ammissione','da un monte ore compreso tra le 580 e le 596 ore', 'Alari Park Studios', 'Core-T-Zone',
                  'da un monte ore compreso tra le 580 e le 596 ore', 'Direttore Didattico', 'Erasmus, ']

In [111]:
courses_corrected = []
for course in list_insegna:
    if any(word in course for word in omitted_string):
        continue 
    
    if course == 'rocessing':
        courses_corrected.append('Processing') 
    else:
        courses_corrected.append(course)

courses_corrected

["Musica d'Insieme",
 ' Tecnico del Suono',
 ' ',
 'Hardware Component',
 'Informatica',
 'Listening',
 'Miking',
 'Recording',
 "Educazione all'ascolto",
 'D.E.M.O.',
 'Virtual Instruments',
 "Musica d'Insieme",
 'Elementi di Composizione',
 'Tastiere elettroniche: linguaggi',
 'Keyboards Technique',
 'Analog Keys',
 'Arranging for Keyboards',
 'Chord Voicing',
 'Lettura e Repertorio',
 'Logic',
 'Note Legali',
 '\xa0Tecnico del Suono',
 'Diego Maggi',
 'Audio Editing',
 'Mixing',
 'Miking',
 'Surround',
 'Summing',
 'Processing',
 'Mastering',
 'Recording',
 'Component',
 'Arrangiamento e produzione',
 'Vocal Interpretation',
 'Interpretazione',
 'Batteria',
 ' ',
 'Contemporary Drum Styles',
 'Drum Techniques',
 'Rock & Heavy Drum Styles',
 'Chitarra Fingerstyle',
 'Storia della Musica Classica',
 'Teoria e Armonia',
 'Basso',
 'Fundamental for Bass',
 'Contemporary Bass Style',
 'Advanced Techniques',
 'Reading',
 'Fretless',
 'Bass & Drum',
 'Canto',
 'Contemporary Vocal Styles',


In [50]:
print(insegna[4])
# insegna_new = insegna
# insegna_new[4] = insegna[4][2:]
# insegna_new[4]

mposizione Pop Rock 


In [ ]:
corsi = [
    ['Violino Pop/Rock'],
    ["Musica d'Insieme"],
    ['Basso Pop Rock'],
    ['Acustica'],
    ["Musica d'Insieme", 'Elementi di Composizione', 'Programmazione Tastiere', ]

]

In [101]:
text_docenti = soup.find('div', class_='col-md-12 text-docenti')
text_with_insegna = text_docenti.find(string=lambda text: 'insegna' in text)
text_with_insegna

'e insegna:'

In [104]:
[b for b in text_docenti.find_all('b') if b.previous_element]

[<b>Elio e Le Storie Tese</b>,
 <b>Dargen d’Amico</b>,
 <b>Verdena</b>,
 <b>Paola&amp;Chiara</b>,
 <b>Haiducii </b>,
 <b>EXPO 2015, XI Giochi Paraolimpici di Sochi (Russia), Asian Games a Jakarta, XXX Universiadi, FIFA Best Award, 70 Anniversario World Holocaust Forum a Praga, Dubai 2017 World Cup, F1 Milano Festival.</b>,
 <b>Enigma</b>,
 <b>Cinecittà World</b>,
 <b>Acquario di Genova</b>,
 <b>Divina Bellezza</b>,
 <b>Topo Gigio, su Rai YoYo</b>,
 <b>Rocco Tanica</b>,
 <b>Lo Specchio di Lorenzo”</b>,
 <b>RAI</b>,
 <b>Giornata Mondiale della Consapevolezza sull’Autismo</b>,
 <b>RadioDeejay</b>,
 <b>Colorado</b>,
 <b>Cristina d’Avena, Virginia Raffaele, Ubaldo Pantani, Brenda Lodigiani, I Soliti Idioti</b>,
 <b>Quelli che il Calcio</b>,
 <b>Fabrizio de André</b>,
 <b>Mille Anni Ancora</b>,
 <b>Mario Arcari, Giorgio Cordini, Ellade Bandini, Laura Deluca</b>,
 <b>Le Nuvole</b>,
 <b>Raiuno</b>,
 <b>Morgan, Antonella Ruggiero e Mario Venuti</b>,
 <b> Tommaso Leddi, Giulio Capiozzo</b>,
 <b>

In [109]:
text_with_insegna = text_docenti.find(string=lambda text: 'insegna' in text)
if text_with_insegna:
    b = [tag for tag in text_docenti.find_all('b') if tag.previous_element.previous_element.previous_element == text_with_insegna].index

b

<function list.index(value, start=0, stop=9223372036854775807, /)>

## Creazione tabella tempo

In [91]:
first_day = datetime(2025,2,1)
last_day = datetime(2025,2,28)

datetime_list = []

hours_list = [h for h in range(9,13)] + [h for h in range(14,18)]
day_list = [d for d in range(first_day.day, last_day.day + 1)]
month_list = [m for m in range(first_day.month, last_day.month + 1)]
year_list = [y for y in range(first_day.year, last_day.year + 1)]

for y in year_list:
    for m in month_list:
        for d in day_list:
            for h in hours_list:
                current_datetime = datetime(y,m,d,h)
                if current_datetime.weekday() < 5:
                    datetime_list.append(datetime(y,m,d,h))

timetable = pd.DataFrame()
timetable['datetime'] = datetime_list
timetable['year'] = [d.year for d in datetime_list]
timetable['month'] = [d.month for d in datetime_list]
timetable['day'] = [d.day for d in datetime_list]
timetable['hour'] = [d.hour for d in datetime_list]
timetable

,datetime,year,month,day,hour
0,2025-02-03 09:00:00,2025,2,3,9
1,2025-02-03 10:00:00,2025,2,3,10
2,2025-02-03 11:00:00,2025,2,3,11
3,2025-02-03 12:00:00,2025,2,3,12
4,2025-02-03 14:00:00,2025,2,3,14
...,...,...,...,...,...
155,2025-02-28 12:00:00,2025,2,28,12
156,2025-02-28 14:00:00,2025,2,28,14
157,2025-02-28 15:00:00,2025,2,28,15
158,2025-02-28 16:00:00,2025,2,28,16


In [100]:
classrooms = pd.DataFrame()
nr_classrooms = 25
classrooms['id'] = ['cl_'+str(i) for i in range(nr_classrooms)]
classrooms[subjects] = [random.choices([0,1], k=len(subjects)) for _ in range(nr_classrooms)]
classrooms

,id,Basso_Elettrico_Pop_Rock,Batteria_e_Percussioni_Pop_Rock,Canto_Pop_Rock,Chitarra_Pop_Rock,Pianoforte_e_Tastiere_elettroniche_Pop_Rock,Composizione_Pop_ Rock,Tecnico_del_suono
0,cl_0,1,1,0,1,1,0,1
1,cl_1,0,0,1,1,0,1,1
2,cl_2,0,0,0,1,1,0,1
3,cl_3,0,1,0,0,1,1,0
4,cl_4,0,1,1,1,1,0,1
5,cl_5,1,0,0,1,0,1,1
6,cl_6,0,1,1,0,1,1,0
7,cl_7,0,1,1,1,0,1,1
8,cl_8,1,0,1,1,0,0,0
9,cl_9,1,1,0,1,1,1,1
